## Downloading and preprocessing MEN
This notebook downloads and preprocessed the [MEN](https://staff.fnwi.uva.nl/e.bruni/resources/MEN) dataset.

In [10]:
%env URL=https://staff.fnwi.uva.nl/e.bruni/resources/MEN.zip
!wget $URL
!unzip MEN.zip

env: URL=https://staff.fnwi.uva.nl/e.bruni/resources/MEN.zip
--2017-08-10 12:58:05--  https://staff.fnwi.uva.nl/e.bruni/resources/MEN.zip
Resolving staff.fnwi.uva.nl... 146.50.61.62
Connecting to staff.fnwi.uva.nl|146.50.61.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96854 (95K) [application/zip]
Saving to: ‘MEN.zip’

MEN.zip             100%[===================>]  94.58K   206KB/s    in 0.5s    

2017-08-10 12:58:06 (206 KB/s) - ‘MEN.zip’ saved [96854/96854]

Archive:  MEN.zip
   creating: MEN/
  inflating: MEN/licence.txt         
  inflating: MEN/.DS_Store           
   creating: MEN/agreement/
  inflating: MEN/agreement/elias-men-ratings.txt  
  inflating: MEN/agreement/agreement-score.txt  
  inflating: MEN/agreement/marcos-men-ratings.txt  
  inflating: MEN/instructions.txt    
  inflating: MEN/MEN_dataset_lemma_form.test  
  inflating: MEN/MEN_dataset_lemma_form.dev  
  inflating: MEN/MEN_dataset_lemma_form_full  
  inflating: MEN/MEN_dataset_nat

In [11]:
!ls

MEN                     MEN.zip                 preprocessing_men.ipynb


In [14]:
!ls MEN

MEN_dataset_lemma_form.dev    agreement
MEN_dataset_lemma_form.test   instructions.txt
MEN_dataset_lemma_form_full   licence.txt
MEN_dataset_natural_form_full


In [16]:
import pandas as pd

In [18]:
raw_data = 'MEN/MEN_dataset_natural_form_full'
df = pd.read_csv(raw_data, header=None, sep=' ')

In [19]:
df.head()

,0,1,2
0,sun,sunlight,50.0
1,automobile,car,50.0
2,river,water,49.0
3,stairs,staircase,49.0
4,morning,sunrise,49.0


In [21]:
df.columns = ['word1', 'word2', 'similarity_out_of_50']

In [24]:
df['similarity'] = df['similarity_out_of_50'] / 5

In [25]:
outfile = '../men.csv'
df.to_csv(outfile, index=False)

In [26]:
!find . -not -name 'preprocessing_men.ipynb' -print0 | xargs -0 rm --
!rm -rf MEN

rm: "." and ".." may not be removed
rm: ./.ipynb_checkpoints: is a directory
rm: ./MEN: is a directory
rm: ./MEN/agreement: is a directory
